In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext sql

In [ ]:
from dnanalysis import *
import nltk
import os
import string
import re
import matplotlib.pyplot as plt

In [ ]:
%sql sqlite:///data/digitalnippon.sql

In [ ]:
authors = %sql SELECT name FROM author WHERE registered_at IS NOT NULL
authors = [author[0] for author in authors]

In [ ]:
common_words = ["''", "``", "...", "spoiler", "--", "“", "–", "’", "»", "«", "zeile", "spalte", "zelle"]
stopwords = nltk.corpus.stopwords.words("german") + common_words

In [ ]:
def handle_post(post: lhtml.Element):
    element = get_html_element_from_string(post)
    element = remove_author_and_date(element)
    element = remove_elements_with_class("quote", element)
    element = remove_elements_with_class("quotecontent", element)
    return element.text_content()

In [ ]:
def all_posts_to_text(user: str):
    posts = %sql SELECT content FROM post_view WHERE author = :user
    posts = (post[0] for post in posts)
    xml_to_text = map(handle_post, posts)
    text = " ".join(xml_to_text)
    text = re.sub(r'\d+', "", text.lower())
    text = nltk.word_tokenize(text)
    global stopwords
    text = (token for token in text if token not in stopwords and token not in string.punctuation)
    return nltk.Text(text)

In [ ]:
def get_unigram_bigram_trigram(user: str):
    unigram = all_posts_to_text(user)
    bigram = nltk.bigrams(unigram)
    bigram = nltk.Text(" ".join(two_words) for two_words in bigram)
    trigram = nltk.trigrams(unigram)
    trigram = nltk.Text(" ".join(three_words) for three_words in trigram)
    return unigram, bigram, trigram

In [ ]:
unigram, bigram, trigram = get_unigram_bigram_trigram("Kasu")

In [ ]:
plt.figure(figsize=(15, 5))
unigram.plot(50)

In [ ]:
plt.figure(figsize=(15, 5))
bigram.plot(50)

In [ ]:
plt.figure(figsize=(15, 5))
trigram.plot(50)